Code below writes .in input files for lammps simulation. Update parameters as desired.

In [ ]:
%%writefile input.in

log             log.logfile
units           real
atom_style      full
dimension        3
newton           on
boundary        p p p


bond_style hybrid harmonic
angle_style hybrid harmonic
dihedral_style hybrid fourier
improper_style cvff
special_bonds lj 0.0 0.0 0.5 coul 0.0 0.0 0.8333333333

# Non-bonded interactions in Sage force field
pair_style lj/cut/coul/long 9.0 9.0
pair_modify mix arithmetic tail yes

# Load the input system from .data file
read_data lammps_files/lammps_input_data.data

# Thermostat and velocity generation
fix 3 all npt temp 298.0 298.0 100.0 iso 1.0 1.0 300.0
velocity all create 298.0 1 rot yes dist gaussian # Change seed for repeat runs (eg from 1 to 2)

# Output control
dump lammpstrj_dump all atom 100 lammps_output_lammpstrj.lammpstrj
thermo_style custom step etotal pe ke temp press lx ly lz vol density
thermo 100

# PME electrostatics in Sage force field
kspace_style pppm 1e-5

# Preliminary energy minimisation to bring atoms to a more sensible state
minimize 1.0e-4 1.0e-6 100 1000 

# Run for X steps at 1 fs δt, with velocity-verlet algorithm
timestep 1
run_style          verlet
run 10000

unfix 3

# Output
compute pe all pe/atom
compute pair all pe/atom pair
compute bond all pe/atom bond
compute angle all pe/atom angle
compute dihedral all pe/atom dihedral
compute improper all pe/atom improper


For short simulations, the simulation can run on a personal computer using the code box below. Otherwise, simulations should be submitted on the cluster via job submission files.

In [ ]:
#box to run the simulation
!lmp < input.in